<a href="https://colab.research.google.com/github/Pranjal001/DLAssignment2_CNN/blob/main/DLA2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch.nn as nn
from torch.nn import functional as funct
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os
from PIL import Image
from torch import optim
import torch
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

In [12]:
class CNN():

  def __init__(self,settings):
    # super().__init__()
    input_channel = settings['input_channel']
    output_size = settings['output_size']
    total_filters = settings['total_filters']
    filter_size = settings['filter_size']

    filter_list_out = self.organise_filter_function(settings['filter_organisation'],filter_size)

    stride_filter = settings['stride']
    padding_filter = settings['padding']
    filter_pool = settings['filter_pool']
    padding_pool = settings['padding_pool']
    stride_pool = settings['stride_pool']
    activation = settings['activation']

    activation = self.activation_call(activation)

    dense_layer_size = settings['dense_layer_size']
    image_size = settings['image_size']

# generating convolution layer and pooling layers along with the dense layer with appropriate sizes

    #-------------------------------------------------------------------------------CONV1--------------------------------------------------------------------------------------------

    self.conv1 = nn.Conv2d(in_channels = input_channel, out_channels =filter_list_out[0], kernel_size = filter_size, padding = padding_filter, stride = stride_filter),
    self.activation1 = activation
    self.pool1 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #--------------------------------------------------------------------------------CONV2---------------------------------------------------------------------------------------

    self.conv2 = nn.Conv2d(in_channels = filter_list_out[0], out_channels =filter_list_out[1], kernel_size = filter_size, padding = padding_filter, stride = stride_filter),
    self.activation2 = activation
    self.pool2 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #---------------------------------------------------------------------------------CONV3------------------------------------------------------------------------------------------

    self.conv3 = nn.Conv2d(in_channels = filter_list_out[1], out_channels =filter_list_out[2], kernel_size = filter_size, padding = padding_filter, stride = stride_filter),
    self.activation3 = activation
    self.pool3 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #----------------------------------------------------------------------------------CONV4-----------------------------------------------------------------------------------------

    self.conv4 = nn.Conv2d(in_channels = filter_list_out[2], out_channels =filter_list_out[3], kernel_size = filter_size, padding = padding_filter, stride = stride_filter),
    self.activation4 = activation
    self.pool4 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #-----------------------------------------------------------------------------------CONV5----------------------------------------------------------------------------------------

    self.conv5 = nn.Conv2d(in_channels = filter_list_out[3], out_channels = filter_list_out[4], kernel_size = filter_size, padding = padding_filter, stride = stride_filter),
    self.activation5 = activation
    self.pool5 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #-----------------------------------------------------------------------------------DENSE_LAYER-------------------------------------------------------------------------------------

    self.flatten = nn.flatten()
    self.FC1 = nn.linear(filter_list_out[4]*image_size*image_size,dense_layer_size)
    self.op_layer = nn.linear(dense_layer_size,output_size)

#=============================================================================== Utility functions ========================================================================================

  def compute_conv_size(W, F, S, P, DP, FP, PP, SP):
    W =  (W - F + 2*P)//S + 1
    return (W + 2*PP - (DP* (FP-1)) - 1)//SP + 1

  def organise_filter_function(self, org_type, filter_size):

    if(org_type == 'same'):
      factor = 1
    if(org_type == 'double'):
      factor = 2
    if(org_type == 'half'):
      factor = 0.5

    filter_list = []
    for i in range(5):
      filter_list.append(filter_size)
      filter_size = int(filter_size*factor)

    return filter_list

  def activation_call(self, act_type):

    if act_type == 'ReLU':
      return nn.ReLU()
    if act_type == 'GELU':
      return nn.GELU()
    if act_type == 'SiLU':
      return nn.SiLU()
    if act_type == 'Mish':
      return nn.Mish()

  def forward(self,data):

    data = self.conv1(data)
    data = self.activation1(data)
    data = self.pool1(data)

    data = self.conv2(data)
    data = self.activation2(data)
    data = self.pool2(data)

    data = self.conv3(data)
    data = self.activation3(data)
    data = self.pool3(data)

    data = self.conv4(data)
    data = self.activation4(data)
    data = self.pool4(data)

    data = self.conv5(data)
    data = self.activation5(data)
    data = self.pool5(data)

    data = self.flatten(data)
    data = self.FC1(data)
    data = self.op_layer(data)
    data = funct.softmax(data,dim = 1)

    return data




In [13]:
settings = {
    'input_channel' : 3,
    'output_size' : 10,
    'data_augmentaion' : "No", # "Yes", "No"
    'batch_normalization' : "No", # "Yes", "No"
    'dropout' : [0.2, 0.3],
    'filter_organisation' : 'same', #'same', 'double', 'half'
    'total_filters': 8, #total filters on each layer
    'filter_size' : 3, #each filter size
    'stride' : 1, #stride value for filter
    'padding' : 0, #padding value
    'filter_pool': 3, #pooled filter size(filter_pool_size not needed)
    'stride_pool' : 1, #stride value during pooling
    'padding_pool' : 0, #padding value for pooling
    'activation' : 'ReLU', # ReLU, GELU, SiLU, Mish,
    'dense_layer_size' : 16,
    'batch_size' : 8,
    'image_size' : 256,
    'epochs':2,
}



In [3]:
print(int(1*0.5))

0


In [9]:
def organise_filter_function( org_type, total_filters , filter_size):

    if(org_type == 'same'):
      factor = 1
    if(org_type == 'double'):
      factor = 2
    if(org_type == 'half'):
      factor = 0.5


    filter_list = []
    for i in range(5):
      filter_list.append(filter_size)
      filter_size = int(filter_size*factor)

    return filter_list

print(organise_filter_function( 'half', 5 , 10))


[10, 5, 2, 1, 0]
